In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:

from src.data_processing.processors.guppy_processors.experimental_metadata import ExperimentMetaData, MetaDataFactory
from src.data_processing.processors.guppy_processors.config_loader import ConfigLoader
from src.data_processing.processors.guppy_processors.data_preprocessor import BatchPreprocessor
from src.data_processing.processors.guppy_processors.processing_strategies import ProcessingStrategy, BehaviorProcessingStrategy



EXPERIMENTAL_CONFIGS = '/Volumes/fsmresfiles/Basic_Sciences/Phys/Lerner_Lab_tnl2633/Shudi/LHA_dopamine/conf/config.yaml'
configs = ConfigLoader(EXPERIMENTAL_CONFIGS)
meta_data_factory = MetaDataFactory(configs)

In [24]:
configs.data_path

PosixPath('/Volumes/fsmresfiles/Basic_Sciences/Phys/Lerner_Lab_tnl2633/Shudi/LHA_dopamine/analyzed_photometry/cohort_1')

In [16]:
behavior_strategy_params={
    'config_key': 'behavioral_events',
    'time_window': (-10, 20),
    'events': (('cue', 'lick'), ('cue', 'encoder'))

}
meta_data_factory = MetaDataFactory(configs)
behavior_strategy = BehaviorProcessingStrategy(**behavior_strategy_params)
batch_preprocessor = BatchPreprocessor(meta_data_factory, behavior_strategy)

In [18]:
batch_preprocessor.process_data(num_processors=4)


event ('cue', 'lick') not found for 4758-231109-120106 
event ('cue', 'lick') not found for 4758-231031-142543 
event ('cue', 'lick') not found for 4755-231109-103518 
event ('cue', 'lick') not found for 4755-231110-104321 
event ('cue', 'lick') not found for 4758-231102-120848 
event ('cue', 'lick') not found for 4758-231121-111532 
event ('cue', 'lick') not found for 4758-231205-114434 
event ('cue', 'lick') not found for 4755-231116-095551 
event ('cue', 'lick') not found for 4755-231205-102542 
event ('cue', 'lick') not found for 4755-231114-130836 
event ('cue', 'lick') not found for 4755-231130-115612 
event ('cue', 'lick') not found for 4755-231103-120816 
event ('cue', 'lick') not found for 4758-231201-130514 
event ('cue', 'lick') not found for 4758-231114-142535 
event ('cue', 'encoder') not found for 4758-231114-142535 
event ('cue', 'lick') not found for 4755-231117-114252 
event ('cue', 'lick') not found for 4758-231130-131542 
event ('cue', 'lick') not found for 4758-2311

In [54]:
files = configs.data_path.rglob('*processed_data.parquet')

In [74]:
from src.data_processing.processors.guppy_processors.aggregators import BehaviorAggregationStrategy, aggregate_data
aggregate_data(configs, BehaviorAggregationStrategy)

In [130]:
df = pd.read_parquet(
    '/Volumes/fsmresfiles/Basic_Sciences/Phys/Lerner_Lab_tnl2633/Shudi/LHA_dopamine/analyzed_photometry/cohort_1/aggregated_data/aggregated_behavior_data.parquet')
df

,lick_aligned_to_cue,encoder_aligned_to_cue,subject,user,date,time
0,0.151515,35.000000,4756.0,User,2023-11-22,2023-12-15 12:10:00
1,0.000000,36.666667,4756.0,User,2023-11-22,2023-12-15 12:10:00
2,0.000000,41.666667,4756.0,User,2023-11-22,2023-12-15 12:10:00
3,0.000000,36.969697,4756.0,User,2023-11-22,2023-12-15 12:10:00
4,0.000000,37.878788,4756.0,User,2023-11-22,2023-12-15 12:10:00
...,...,...,...,...,...,...
150,NaN,6.896552,4758.0,User,2023-11-16,2023-12-15 11:25:06
151,NaN,7.931034,4758.0,User,2023-11-16,2023-12-15 11:25:06
152,NaN,6.206897,4758.0,User,2023-11-16,2023-12-15 11:25:06
153,NaN,6.551724,4758.0,User,2023-11-16,2023-12-15 11:25:06
